# Save and load models
- Learn how to save and load models using TensorFlow Keras API. 
- Full tutorial reference: https://www.tensorflow.org/tutorials/keras/save_and_load

### Import TensorFlow and necessary libraries

In [1]:
import os
import tensorflow as tf

print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.9.1


### Get an example dataset

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# for fast implementation, use the first 1,000 examples
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [3]:
train_images.shape

(1000, 784)

### Define a model

In [4]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10)
    ])
    
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )
    
    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### Checkpoint callback usage
- The *tf.keras.callbacks.ModelCheckpoint()* callback saves weights only during training.

In [5]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1
)

model.fit(
    train_images,
    train_labels,
    epochs=10,
    validation_data=(test_images, test_labels),
    callbacks=[cp_callback]
)

Epoch 1/10
26/32 [=======================>......] - ETA: 0s - loss: 1.2753 - sparse_categorical_accuracy: 0.6250
Epoch 1: saving model to training_1\cp.ckpt
32/32 [==============================] - 2s 25ms/step - loss: 1.1747 - sparse_categorical_accuracy: 0.6560 - val_loss: 0.7280 - val_sparse_categorical_accuracy: 0.7800
Epoch 2/10
29/32 [==========================>...] - ETA: 0s - loss: 0.4147 - sparse_categorical_accuracy: 0.8815
Epoch 2: saving model to training_1\cp.ckpt
32/32 [==============================] - 1s 19ms/step - loss: 0.4181 - sparse_categorical_accuracy: 0.8810 - val_loss: 0.5280 - val_sparse_categorical_accuracy: 0.8380
Epoch 3/10
29/32 [==========================>...] - ETA: 0s - loss: 0.2878 - sparse_categorical_accuracy: 0.9224
Epoch 3: saving model to training_1\cp.ckpt
32/32 [==============================] - 1s 18ms/step - loss: 0.2820 - sparse_categorical_accuracy: 0.9230 - val_loss: 0.4591 - val_sparse_categorical_accuracy: 0.8520
Epoch 4/10
28/32 [=======

In [6]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

### Load the weights from the checkpoint on a new model

In [7]:
# create a new model instance
new_model = create_model()

# Evaluate the model
new_model_eval = new_model.evaluate(test_images, test_labels, verbose=2)
print('New model sparse_categorical_accuracy:', new_model_eval[1])

32/32 - 0s - loss: 2.3079 - sparse_categorical_accuracy: 0.1130 - 231ms/epoch - 7ms/step
New model sparse_categorical_accuracy: 0.11299999803304672


In [8]:
# load the weights
new_model.load_weights(checkpoint_path)

# re-evaluate the model
new_model_eval = new_model.evaluate(test_images, test_labels, verbose=2)
print('New model sparse_categorical_accuracy:', new_model_eval[1])

32/32 - 0s - loss: 0.4118 - sparse_categorical_accuracy: 0.8680 - 122ms/epoch - 4ms/step
New model sparse_categorical_accuracy: 0.8679999709129333


### Save the entire model
- An entire model can be saved in two different format:
    - SaveModel (default file format in TensorFlow2)
    - HDF5

In [9]:
# save the entire model as a SaveModel file format
!mkdir -p saved_model
new_model.save('saved_model')

INFO:tensorflow:Assets written to: saved_model\assets


In [12]:
# reload a model from saved model
novel_model = tf.keras.models.load_model('saved_model')
novel_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 512)               401920    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [13]:
# evaluate the model; it ends up showing the same accuracy as new_model since novel_model reloads the same model
novel_model_eval = novel_model.evaluate(test_images, test_labels, verbose=2)
print('Novel model sparse_categorical_accuracy:', novel_model_eval[1])

32/32 - 0s - loss: 0.4118 - sparse_categorical_accuracy: 0.8680 - 266ms/epoch - 8ms/step
Novel model sparse_categorical_accuracy: 0.8679999709129333


In [15]:
# save the entire model as a HDF5 file format; the '.h5' extension indicates the model should be saved as HDF5
new_model.save('hdf5_model.h5')

In [16]:
hdf5_model = tf.keras.models.load_model('hdf5_model.h5')
hdf5_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 512)               401920    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [17]:
# evaluate the model
hdf5_model_eval = hdf5_model.evaluate(test_images, test_labels, verbose=2)
print('HDF5 model sparse_categorical_accuracy:', hdf5_model_eval[1])

32/32 - 0s - loss: 0.4118 - sparse_categorical_accuracy: 0.8680 - 239ms/epoch - 7ms/step
HDF5 model sparse_categorical_accuracy: 0.8679999709129333


In [18]:
# MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.